In [17]:
import numpy as np
import pandas as pd
import re

np.set_printoptions(threshold=100000)
pd.options.display.max_rows = 500
pd.options.display.max_columns = 100
pd.set_option('expand_frame_repr', False)

df_all = pd.read_excel(r'e:/GIS/土地供应2005_2018_地表_全口径_宗地.xlsx')
df = df_all[(df_all.GB_DLMC=='住宅用地')| (df_all.GB_DLMC=='工矿仓储用地') |(df_all.GB_DLMC=='商服用地') | (df_all.GB_DLMC=='工业用地')]
df = df[['PARCEL_NO', 'YDMJ','JZMJ', 'HT_DLMC', 'GB_DLMC', '建筑面积说明']]

col_dict = [
    {'name_en': 'HT_DLMC', 'name_zh': '合同_地类名称'},
    {'name_en': 'GB_DLMC', 'name_zh': '国标_地类名称'},
    {'name_en': 'ZDH', 'name_zh': '宗地号'},
    {'name_en': 'YDMJ', 'name_zh': '用地面积'},
    {'name_en': 'JZMJ', 'name_zh': '建筑面积'},
    
]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3460 entries, 0 to 4811
Data columns (total 6 columns):
PARCEL_NO    3460 non-null object
YDMJ         3438 non-null float64
JZMJ         3438 non-null float64
HT_DLMC      3440 non-null object
GB_DLMC      3460 non-null object
建筑面积说明       3368 non-null object
dtypes: float64(2), object(4)
memory usage: 189.2+ KB


In [2]:
# df = df[df.PARCEL_NO=='G04310-0032']

In [18]:
def get_list_num(df, p_str_list, c, column_name):
    pub = '共?(\d+(\.\d+)?)(平方|平方米|平米|㎡)'
    list_sum = 0
    re_list = []
    for i in p_str_list:
        pt = re.findall(re.compile(i + pub), c)
        if pt:
            num = 0
            for j in range(0, len(pt)):
                num = num + float(pt[j][0])
                re_list.append(''.join(pt[j]))
            list_sum = list_sum + num
            for j in pt:
                jj = ''.join(j)
                c = re.sub(jj, '', c)
        pt2 = re.findall(re.compile(pub + '的' + i), c)
        if pt2:
            num2 = 0
            for k in range(0, len(pt2)):
                num2 = num2 + float(pt2[k][0])
                re_list.append(''.join(pt2[k]))
            list_sum = list_sum + num2
        for rr in re_list:
            c = c.replace(rr, '')
    df.at[index, column_name] = list_sum

for index, rows in df.iterrows():
    replace_list = [
        
        
    ]
    c = str(rows['建筑面积说明']).replace(' ', '').replace('\n', '').replace('\r', '').replace('面积', '').replace('用房', '')
    c = c.replace(':', '').replace('：', '').replace('中心', '').replace('不少于', '').replace('不超过', '').replace('场地', '').replace('用地', '')
    c = c.replace('建筑' , '').replace('总', '').replace('≤', '').replace('≥', '').replace('业务', '').replace(r'[独立占地]', '')
    c = c.replace('【', '').replace('】', '').replace('楼', '').replace('大楼', '').replace('总计', '')
    c = re.sub(r'（\D+）', '', c)
    
    p_dict = [
        {'col': '住宅-总', 'p': ['居住', '住宅', '[^保障性]住房']},
        {'col': '住宅-公共租赁住房', 'p': ['公共租赁住房', '保障性住房', '保障性用房']},
        {'col': '住宅-安居型商品房', 'p': ['安居型商品房']},
        {'col': '住宅-人才住房', 'p': ['人才住房']},
        
        {'col': '产业-总', 'p': ['产业', '厂房及配套', '产业研发', '产业生产']},
        {'col': '产业-工业用房', 'p': ['厂房', '设备', '车间', '仓库', '物流建筑']},
        {'col': '产业-创新型产业用房', 'p': ['创新型产业', '科研']},
        {'col': '产业配套用房-总', 'p': ['配套商业及行政办公', '配套综合服务']},
        {'col': '产业配套用房-宿舍食堂', 'p': ['宿舍', '食堂', '值班休息', '宿舍及配套']},
        {'col': '产业配套用房-配套办公', 'p': ['配套办公', '办公配套', '配套行政办公']},
        {'col': '产业配套用房-配套商业', 'p': ['配套商业', '商业配套','配套商业设施', '配套商业服务设施']},
        
        {'col': '商业', 'p': ['[^配套]商业', '[^配套]商业设施', '[^配套]商业服务设施', '^商业', '^商业设施', '^商业服务设施']},
        {'col': '办公', 'p': ['[^配套]办公', '^办公']},
        
        {'col': '商务公寓', 'p': ['商务公寓']},
        {'col': '人才公寓', 'p': ['人才公寓']},
        {'col': '服务业', 'p': ['服务业', '文化设施', '会所', '剧院', '美术馆', '剧院及其配套', '地上游乐设施及相关配套', '书城']},
        
        {'col': '酒店', 'p': ['酒店']},
        
        {'col': '公共设施配套-总', 'p': ['公共配套设施', '公共配套', '公建配套设施', '公建配套']},
        {'col': '公共设施配套-社区管理', 'p': ['居委会', '社区服务站', '警务室', '便民服务站']},
        {'col': '公共设施配套-社区文化体育', 'p': [ '文化活动室', '社区体育活动', '活动站', '文体活动', '文化室', '文化娱乐',
                                       '社区运动', '文化活动']},
        {'col': '公共设施配套-社区健康', 'p': ['社区健康服务',  '社康','康体', '康复']},
        {'col': '公共设施配套-养老', 'p': [ '照料', '老年活动', '照料服务']},
        {'col': '公共设施配套-生活', 'p': ['菜市场', '充电站']},
        {'col': '公共设施配套-幼儿园', 'p': ['幼儿园']},
        {'col': '公共设施配套-学校', 'p': ['学校']},
        {'col': '公共配套设施-卫生', 'p': ['回收点','回收站', '垃圾站', '垃圾收集站', '垃圾处理站', '污水处理站','垃圾转运站', '公厕', '厕所']},
        {'col': '公共配套设施-交通邮政', 'p': ['公交场站', '公交站场', '公交站','换乘站', '公交首末站',  '公交运营', '公交充电站' 
                                    '邮政局','邮政所', '邮电设施', '综合车站']},
        
        {'col': '配套设施-停车场', 'p': ['停车场', '停车库']},
        {'col': '配套设施-物业', 'p':['物管', '物业服务', '物业', '物业管理']}
    ]
    
    for i in p_dict:
        get_list_num(df, i['p'], c, i['col'])

df

,PARCEL_NO,YDMJ,JZMJ,HT_DLMC,GB_DLMC,建筑面积说明,住宅-总,住宅-公共租赁住房,住宅-安居型商品房,住宅-人才住房,产业-总,产业-工业用房,产业-创新型产业用房,产业配套用房-总,产业配套用房-宿舍食堂,产业配套用房-配套办公,产业配套用房-配套商业,商业,办公,商务公寓,人才公寓,服务业,酒店,公共设施配套-总,公共设施配套-社区管理,公共设施配套-社区文化体育,公共设施配套-社区健康,公共设施配套-养老,公共设施配套-生活,公共设施配套-幼儿园,公共设施配套-学校,公共配套设施-卫生,公共配套设施-交通邮政,配套设施-停车场,配套设施-物业
0,A001-0126,9.280828e+04,246870.00,居住用地,住宅用地,其中:居住 232620平方米、商业6000 平方米（含净菜市场1000平方米）、会所400...,232620.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,6000.00,0.00,0.0,0.0,4000.00,0.0,4250.0,220.0,1500.0,400.0,0.0,1000.00,1600.0,0.0,230.0,100.0,0.00,0.0
10,A002-0012,6.648630e+03,24600.00,商业性办公用地,商服用地,办公面积20600平方米，商业面积4000平方米。地下车库，设备用房，公共通道，不计容积率。,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,4000.00,20600.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0
11,A002-0013,6.027980e+03,20800.00,商业性办公用地,商服用地,办公面积17800平方米，商业面积3000平方米。地下车库，设备用房，公共通道，不计容积率。,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,3000.00,17800.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0
12,A002-0020,5.440390e+03,23230.00,商业性办公用地,商服用地,办公面积19730平方米，商业面积3500平方米。地下车库，设备用房，公共通道，不计容积率。,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,3500.00,19730.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0
13,A002-0037,5.430820e+03,32580.00,商业服务业设施用地,商服用地,其中商业性办公30180平方米，商业2350平方米，警务室50平方米。,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,2350.00,30180.00,0.0,0.0,0.00,0.0,0.0,50.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0
14,A002-0038,6.202070e+03,38450.00,商业服务业设施用地,商服用地,商业性办公35450平方米，商业3000平方米。,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,3000.00,35450.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0
16,A002-0040,2.117467e+04,84600.00,商业服务业设施用地,商服用地,酒店面积49500平方米（客房数≥400间），美术馆500平方米，商务公寓34600平方米，...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,34600.0,0.0,500.00,49500.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0
18,A002-0042,7.665640e+03,55190.00,商业服务业设施用地,商服用地,商业性办公46500平方米，商业8690平方米。,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,8690.00,46500.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0
19,A002-0043,4.602980e+03,55230.00,商业用地,商服用地,其中：商业5060平方米，商业性办公50000平方米，物业管理物业服务用房170平方米。\n...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,5060.00,50000.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,170.0
20,A002-0046,2.072304e+04,167860.00,商业性办公用地,商服用地,其中，商业15000平方米、商业性办公152760平方米、公共厕所100平方米。,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,15000.00,152760.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,100.0,0.0,0.00,0.0


In [41]:
df2 = df[df.PARCEL_NO=='G04310-0032']
df2

,PARCEL_NO,YDMJ,JZMJ,HT_DLMC,GB_DLMC,建筑面积说明,住宅-总,住宅-公共租赁住房,住宅-安居型商品房,住宅-人才住房,产业-总,产业-工业用房,产业-创新型产业用房,产业配套用房-总,产业配套用房-宿舍食堂,产业配套用房-配套办公,产业配套用房-配套商业,商业,办公,商务公寓,人才公寓,服务业,酒店,公共设施配套-总,公共设施配套-社区管理,公共设施配套-社区文化体育,公共设施配套-社区健康,公共设施配套-养老,公共设施配套-生活,公共设施配套-幼儿园,公共设施配套-学校,公共配套设施-卫生,公共配套设施-交通邮政,配套设施-停车场,配套设施-物业
2280,G04310-0032,83713.78,241704.99,一类工业用地,工矿仓储用地,厂房226326平方米，检测大楼8070平方米，配套商业及行政办公37998平方米，综合服务...,0.0,0.0,0.0,0.0,0.0,226326.0,0.0,50228.0,50228.0,0.0,0.0,0.0,37998.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
pd.DataFrame.to_excel(df, 'E:/GIS/tt5.xlsx')

In [35]:
df.drop(['YDMJ','JZMJ', 'HT_DLMC', 'GB_DLMC', '建筑面积说明'], axis=1, inplace=True)


ValueError: labels ['YDMJ' 'JZMJ' 'HT_DLMC' 'GB_DLMC' '建筑面积说明'] not contained in axis

In [40]:
df_c = pd.merge(df_all, df, on='PARCEL_NO', how='left')
pd.DataFrame.to_excel(df_c, 'E:/GIS/tz.xlsx', index=None)